In [1]:
# -*- coding: utf-8 -*-
from src import Main
from src.preprocessing import Preprocessing
from src import Word2VecModel
from src import ClusterModel

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import pandas as pd
import csv

%matplotlib inline

In [2]:
df_bus = pd.read_json('dataset/business.json', lines=True)

In [3]:
def Get_cluster_labels(size=20):
    thefile = open('dataset/300ktop_category_' + str(size) + '.txt', 'r')
    reader = csv.reader(thefile)
    labels = [] # This will store the labels
    for row in reader:
        labels.append(row[0])
    thefile.close()
    labels = np.array(labels)
    return labels

def Get_data():
    df = pd.read_json('dataset/processed_restaurant_reviews_1.json')
    data = df.text.apply(' '.join)[:300000]
    del(df)
    return data

def predictTopic(InputString, countvec, lda):
    processed_review = Preprocessing(InputString, 1)
    cv = vec.transform([' '.join(processed_review)])
    proba = lda.transform(cv)[0]
    first = np.argsort(lda.transform(cv)[0])[::-1][0]
    second = np.argsort(lda.transform(cv)[0])[::-1][1]
    return (first, proba[first]), (second, proba[second])

def getHiddenRatings(business_id, df):
    reviews = df[df['business_id'] == business_id]
    business_name = df_bus.loc[df_bus['business_id']==business_id, 'name'].asobject[0]
    average = reviews.stars.mean()
    reviews.groupby('labels')
    service_quality = reviews[(reviews['labels'] == '0')|(reviews['labels'] == '4')|(reviews['labels'] == '19')].stars.mean()
    food_quality = reviews[(reviews['labels'] == '7')|(reviews['labels'] == '14')].stars.mean()
    price_quality = reviews[(reviews['labels'] == '17')].stars.mean()
    print "Restaurant: " + business_name
    print "Total # of reviews: " + str(reviews.shape[0])
    print "Overall Average: " + str(average) 
    print "Service Average: " + str(service_quality)
    print "Food Average: " + str(food_quality)
    print "Pricing Average: " + str(price_quality)
#     return average
    

In [4]:
word2vec_model = Word2VecModel.load('models/word2vec_model50')
df = pd.read_json('dataset/processed_restaurant_reviews_1.json')
df = df[:300000]
data = Get_data()
labels = Get_cluster_labels(20)
df['labels'] = labels
lda = ClusterModel.load('models/lda_20.pkl')
vec = ClusterModel.load('models/countvectorizer.pkl')
feature_names = ClusterModel.load('models/tf_feature_names.pkl')
by_topic = df.groupby('labels').mean()
w2vSimilar = word2vec_model.wv.most_similar

In [14]:
df_bus = df_bus[[('Restaurants' in categories) for categories in df_bus['categories']]]

In [16]:
df_bus.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state
4,581 Howe Ave,"{u'Alcohol': u'full_bar', u'HasTV': True, u'No...",PfOCPjBrlQAnz__NXj9h_w,"[American (New), Nightlife, Bars, Sandwiches, ...",Cuyahoga Falls,"{u'Monday': u'11:00-1:00', u'Tuesday': u'11:00...",1,41.119535,-81.475690,Brick House Tavern + Tap,,44221,116,3.5,OH
5,Richterstr. 11,"{u'GoodForMeal': {u'dessert': False, u'latenig...",o9eMRCWt5PkpLDE0gOPtcQ,"[Italian, Restaurants]",Stuttgart,"{u'Monday': u'18:00-0:00', u'Tuesday': u'18:00...",1,48.727200,9.147950,Messina,,70567,5,4.0,BW
10,737 West Pike St,"{u'GoodForMeal': {u'dessert': False, u'latenig...",XOSRcvtaKc_Q5H1SAzN20A,"[Breakfast & Brunch, Gluten-Free, Coffee & Tea...",Houston,{},0,40.241548,-80.212815,East Coast Coffee,,15342,3,4.5,PA
14,600 E 4th St,"{u'GoodForMeal': {u'dessert': False, u'latenig...",fNMVV_ZX7CJSDWQGdOM8Nw,"[Restaurants, American (Traditional)]",Charlotte,"{u'Tuesday': u'7:00-15:00', u'Friday': u'7:00-...",1,35.221647,-80.839345,Showmars Government Center,Uptown,28202,7,3.5,NC
15,2459 Yonge St,"{u'GoodForMeal': {u'dessert': False, u'latenig...",l09JfMeQ6ynYs5MCJtrcmQ,"[Italian, French, Restaurants]",Toronto,"{u'Monday': u'9:00-22:00', u'Tuesday': u'9:00-...",0,43.711399,-79.399339,Alize Catering,Yonge and Eglinton,M4P 2H6,12,3.0,ON


In [5]:
# Display top words from each topic
# ClusterModel.display_topics(lda, feature_names, 30)

In [6]:
# Create word clouds at '/images'
Main.Make_word_cloud(data, labels)

In [7]:
allData = []
sizes = [10, 15, 20, 25, 30, 40]
for i in range(6):
    thefile = open('dataset/300ktop_category_' + str(sizes[i]) + '.txt', 'r')
    reader = csv.reader(thefile)
    labels = np.zeros(40) # This will store the labels
    for row in reader:
        labels[int(row[0])]+=1
    thefile.close()
    allData.append(labels.copy())
ldaClusterChart = pd.DataFrame(allData, index = sizes).T

In [17]:
review = u"Meh is right. I saw the lines out the door. I saw the hype online and I'm sorely disappointed. We came here on a Thursday at around 6pm. Luckily there was no wait. For an appetizer we ordered the takoyaki. I'm not sure if the texture is supposed to be that way, but the exterior was crispy, but the inside was like mush. I was not expecting that. I didn't like the texture at all. For the main course I ordered the pot belly ramen with a chasu bowl, while my other two guests got the miso ramen. All of us agreed that the pot belly ramen was better than the miso. The broth was rich in flavor, more so than the miso ramen. The miso one was just spicy in the initial sip, but was not consistent throughout. I also didn't like the noodles. It was too skinny and didn't hold the flavor of the broth very well. If you like the Vietnamese entree thit kho (braised pork belly), you'll like it. Tasted just like that. The place is a little overpriced for the portion too. I was hoping for it to be good so I didn't have to go all the way to Orenchi. Nope, I'll not be coming back"
review = u"I don't know how I've never reviewed kickin crab lol?! Definitely the best place for cajun/seafood IMO. I especially like this place in comparison to places like boiling crab because I feel like the service is a lot better, the sauce is tastier, and I just feel like the seafood is a lot more fresh which is super important! I tend to go for one of the combo deals since I'm usually with a group and we can all share it for a good deal. GARLIC NOODLES! Oh my god the best I've ever had. You have to order garlic noodles when you come, there's no exceptions.... The only thing that sucks would be the parking. If you come during peak dinner hours...or actually any hours...be prepared to be circling the lot a few times. The spaces are really small too. Good luck!"
review = u"milkshake"
review = u"I came here last Saturday for brunch with my husband and MIL to celebrate my birthday, and I was SUPER excited about it! I have dined here many times, just never for brunch and the food and service have always been fantastic. This visit was a COMPLETELY different story. We sat down on the patio since it was a beautiful day and I love the relaxing environment of the outdoors during brunch. Our server came to our table, his name was Ellis (I could be spelling this wrong.) Right off the bat, he was straight up RUDE. He seemed very annoyed at the fact that he was at work, or was extremely hungover. We walked up to the table and we ended up greeting him before he greeted us, because he just looked at us and gave us a look like the fuck do you people want? It was kinda strange but we tried to brush it off. He then said so it's somebodys birthday here? then pointed at my MIL. She laughed and said no, its hers! and pointed to me. He rolled his eyes and said great. And walked away. I couldnt tell at this point if he thought he was being funny or ironic as the asshole waiter but it was getting old fast."
review = u"I've been here a couple times now and have to say this is the BEST taqueria I've been to. Everything is always extremely fresh and DELICIOUS! My go to order is the tacos. I usually Carne Asada, Pollo, and Al Pastor. The meats are always seasoned perfectly and SOOO tender. The taqueria is very clean and they have a great salsa bar that also has fresh fruit."
review = u"fun"
review = u"Melanie is a treasure that must be protected at all costs. Seriously though, she was super-cool and funny and gave us some pretty fun and adventurous recommendations that we would otherwise not have tried. Really attentive, she even helped us cook our food - it felt like we were getting an experience that other restaurant-goers were not getting. Like v.i.p. status. That's how awesome she is. Food was really good. Excellent recommendations. Next time I come here, I'm just going to tell them to surprise me. I like the music here. Will definitely come back."
predictTopic(review, vec, lda)

((0, 0.38162292057044694), (7, 0.31524944509640945))

In [21]:
# Ramen - Japanese + Mexican = Mexican Food
w2vSimilar(positive=['ramen', 'mexican'], 
           negative=['japanese'])

[(u'machaca', 0.7345311641693115),
 (u'posole', 0.7244584560394287),
 (u'pozole', 0.7185253500938416),
 (u'burro', 0.7165212631225586),
 (u'chimichanga', 0.7074536681175232),
 (u'filibertos', 0.7052419781684875),
 (u'queso', 0.7044840455055237),
 (u'carnitas', 0.7036306262016296),
 (u'sopes', 0.7010722160339355),
 (u'chimi', 0.6951133012771606)]

# Yelp Reviews Topic Extraction
### By InHo Rha and Terry Shih
#### Group #6
<br>
##### [https://github.com/edwardrha/YelpReviewPrediction](https://github.com/edwardrha/YelpReviewPrediction)


# Introduction

### Data
 * Yelp Dataset Challenge - 5.2 million reviews from Yelp
 * 3.4 million of which are restaurant reviews
 * We used 300k reviews due to time and hardware limitation

### Unsupervised Topic Extraction
 * Latent Dirichlet allocation
 * Examniation of extracted clusters and manual labeling
 * Attempted use of Word2Vec

### Why?
 * Learn to gain insight into unlabeled text data
 * Find the categorical ratings of restaurants
 * Picking up trends in the text data

## Sample Data

In [79]:
# We removed stop words and lemmatized the text data 
# and then converted them into tokens
df.head(1)

,business_id,date,review_id,stars,text,user_id,labels
0,0W4lkclzZThpx3V65bVgig,2016-05-28,v0i_UHJMo_hPBq9bxWvW4w,5,"[love, staff, love, meat, love, place, prepare...",bv2nCi5Qv5vroFiqKGopiw,0


# Latent Dirichlet allocation

### What is it?
 * An unsupervised clustering algorithm
 * No label needed, only 'k'

### Why LDA?
 * Allows us to extract multiple subtopics from a corpus
 * Promising tool for topic modeling

In [10]:
ldaClusterChart

,10,15,20,25,30,40
0,92986.0,108219.0,87285.0,88150.0,46595.0,16348.0
1,24975.0,2907.0,18060.0,6044.0,3844.0,10463.0
2,2125.0,134.0,1533.0,19.0,8.0,1861.0
3,25187.0,1753.0,6372.0,6892.0,2078.0,139.0
4,51113.0,1807.0,21489.0,10840.0,338.0,46.0
5,30190.0,27101.0,23170.0,9465.0,310.0,173.0
6,2567.0,927.0,424.0,1191.0,22.0,33.0
7,52421.0,70328.0,85854.0,52506.0,13008.0,44394.0
8,2630.0,133.0,48.0,17.0,19.0,40.0
9,15806.0,32521.0,13513.0,2128.0,3812.0,8563.0


# Manually Labeled Topic Clusters (K = 20)
| Topic         | Label           | Topic         | Label           | 
| :------------ |:--------------- | :------------ |:--------------- | 
| 00            | Service Quality | 10            | Arizona Pasty   | 
| 01            | Dinner          | 11            | Unsure          | 
| 02            | Bars            | 12            | Fancy/Steakhouse| 
| 03            | Breakfast/Cafe  | 13            | Night life      | 
| 04            | Bad Service     | 14            | Positive meat   | 
| 05            | Comfort/Fastfood| 15            | French          | 
| 06            | Montreal Sandwiches  | 16       | Shake shacks/Hotdogs     | 
| 07            | Food Quality    | 17            | Pricing/Mexican | 
| 08            | Drive Thru      | 18            | Desserts      | 
| 09            | Asian           | 19            | Ambience      | 













## Cluster 3 - Breakfast/Cafe
![alt text](images/wordcloud_3.png)

## Cluster 9 - Asian
![alt text](images/wordcloud_9.png)

# Examples

In [11]:
review = u"The ramen came out quickly and the broth was FANTASTIC. I'm a broth person who enjoys her noodles, but the broth has to hold its own. Theirs does! I got pork ramen and regretted nothing."
predictTopic(review, vec, lda)

((9, 0.51335032357099275), (7, 0.16339487225456326))

| Topic         | Label           | Topic         | Label           | 
| :------------ |:--------------- | :------------ |:--------------- | 
| 00            | Service Quality | 10            | Arizona Pasty   | 
| 01            | Dinner          | 11            | Unsure          | 
| 02            | Bars            | 12            | Fancy/Steakhouse| 
| 03            | Breakfast/Cafe  | 13            | Night life      | 
| 04            | Bad Service     | 14            | Positive meat   | 
| 05            | Comfort/Fastfood| 15            | French          | 
| 06            | Montreal Sandwiches  | 16       | Shake shacks/Hotdogs     | 
| 07            | Food Quality    | 17            | Pricing/Mexican | 
| 08            | Drive Thru      | 18            | Desserts      | 
| 09            | Asian           | 19            | Ambience      | 












In [12]:
review = u"Horrible restaurant. Never coming back."
predictTopic(review, vec, lda)

((4, 0.84166666352868491), (0, 0.0083333358824793612))

| Topic         | Label           | Topic         | Label           | 
| :------------ |:--------------- | :------------ |:--------------- | 
| 00            | Service Quality | 10            | Arizona Pasty   | 
| 01            | Dinner          | 11            | Unsure          | 
| 02            | Bars            | 12            | Fancy/Steakhouse| 
| 03            | Breakfast/Cafe  | 13            | Night life      | 
| 04            | Bad Service     | 14            | Positive meat   | 
| 05            | Comfort/Fastfood| 15            | French          | 
| 06            | Montreal Sandwiches  | 16       | Shake shacks/Hotdogs     | 
| 07            | Food Quality    | 17            | Pricing/Mexican | 
| 08            | Drive Thru      | 18            | Desserts      | 
| 09            | Asian           | 19            | Ambience      | 












In [13]:
review = u"Hands down one of the best Mexican places to eat at. Everything from the amazing burritos, to the nachos, tacos and gorditas- are amazing. Everything is made fresh and quick. They have a large variety within the menu so I highly recommend this place"
predictTopic(review, vec, lda)

((0, 0.57205798662973839), (17, 0.39044201299950482))

| Topic         | Label           | Topic         | Label           | 
| :------------ |:--------------- | :------------ |:--------------- | 
| 00            | Service Quality | 10            | Arizona Pasty   | 
| 01            | Dinner          | 11            | Unsure          | 
| 02            | Bars            | 12            | Fancy/Steakhouse| 
| 03            | Breakfast/Cafe  | 13            | Night life      | 
| 04            | Bad Service     | 14            | Positive meat   | 
| 05            | Comfort/Fastfood| 15            | French          | 
| 06            | Montreal Sandwiches  | 16       | Shake shacks/Hotdogs     | 
| 07            | Food Quality    | 17            | Pricing/Mexican | 
| 08            | Drive Thru      | 18            | Desserts      | 
| 09            | Asian           | 19            | Ambience      | 











# Categorical ratings of a restaurant

In [69]:
getHiddenRatings(business_id='6C_8Mh4lmLc_QEs3hHleBg', df=df)

Restaurant: El Pollo Supremo
Total # of reviews: 209
Overall Average: 4.20574162679
Service Average: 4.44444444444
Food Average: 4.0652173913
Pricing Average: 4.05714285714


In [67]:
getHiddenRatings(business_id='Gt4z3AylNTsEPDkzkaC7HA', df=df)

Restaurant: Stacks & Yolks
Total # of reviews: 660
Overall Average: 3.25606060606
Service Average: 3.5406162465
Food Average: 2.46195652174
Pricing Average: 4.05555555556


# Barriers we faced
 * Huge data - Even with Amazon Web Services, we faced limitations
 * Finding adequate clustering algorithms
 * Not enough time/manpower - Labeling, validating

### Word2Vec
![alt text](images/w2v.png)

In [75]:
# (Ramen - Japanese) + Mexican = Mexican Food
word2vec_model.wv.most_similar(positive=['ramen', 'mexican']
                               , negative=['japanese'])

[(u'machaca', 0.7345311641693115),
 (u'posole', 0.7244584560394287),
 (u'pozole', 0.7185253500938416),
 (u'burro', 0.7165212631225586),
 (u'chimichanga', 0.7074536681175232),
 (u'filibertos', 0.7052419781684875),
 (u'queso', 0.7044840455055237),
 (u'carnitas', 0.7036306262016296),
 (u'sopes', 0.7010722160339355),
 (u'chimi', 0.6951133012771606)]

# Thank You
#### References
 * sklearn LDA example code 
 http://scikit-learn.org/0.18/auto_examples/applications/topics_extraction_with_nmf_lda.html
 * Improving Restaurants by Extracting Subtopics from Yelp Reviews 
 https://www.yelp.com/html/pdf/YelpDatasetChallengeWinner_ImprovingRestaurants.pdf
 * Yelp Dataset yelp dataset